<a href="https://colab.research.google.com/github/ahmedhassan97/sentiment-analysis_using_nlp/blob/master/lstm_sentiment_analysis_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

** ** author : ahmed hassan **
   
 githup : https://github.com/ahmedhassan97

 linkedin : https://www.kaggle.com/ahmedhassan97

project to classify Twitter Sentiment into 

two classification using nlp algorithm

data from kaggle 

link : https://www.kaggle.com/crowdflower/first-gop-debate-twitter-sentiment



In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [27]:
#zip file
from zipfile import ZipFile
file_name = "data.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('done')

done


Only keeping the necessary columns.

In [0]:
data = pd.read_csv('Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

Next, I am dropping the 'Neutral' sentiments as my goal was to only differentiate positive and negative tweets. After that, I am filtering the tweets so only valid texts and words remain.  Then, I define the number of max features as 2000 and use Tokenizer to vectorize and convert text into Sequences so the Network can deal with it as input.

In [42]:
data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)
print(data["text"])
for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
print(X[:3])
X = pad_sequences(X)
print(X)
print(X.shape[1])


4472
16986
1          scottwalker didnt catch the full gopdebate l...
3          robgeorge that carly fiorina is trending  ho...
4          danscavino gopdebate w realdonaldtrump deliv...
5          gregabbott_tx tedcruz on my first day i will...
6          warriorwoman91 i liked her and was happy whe...
                               ...                        
13866      cappy_yarbrough love to see men who will nev...
13867      georgehenryw who thought huckabee exceeded t...
13868      lrihendry tedcruz as president i will always...
13869      jrehling gopdebate donald trump says that he...
13870      lrihendry tedcruz headed into the presidenti...
Name: text, Length: 10729, dtype: object
[[363, 122, 1, 703, 2, 39, 58, 237, 37, 210, 6, 174, 1761, 12, 1324, 1409, 743], [16, 284, 252, 5, 821, 102, 167, 26, 136, 6, 1, 173, 12, 2, 233, 724, 17], [1261, 2, 303, 23, 1943, 1, 1632, 216, 12, 1, 704, 6, 185, 207, 371, 670]]
[[   0    0    0 ... 1324 1409  743]
 [   0    0    0 ...  233  724 

Next, I compose the LSTM Network. Note that **embed_dim**, **lstm_out**, **batch_size**, **droupout_x** variables are hyperparameters, their values are somehow intuitive, can be and must be played with in order to achieve good results. Please also note that I am using softmax as activation function. The reason is that our Network is using categorical crossentropy, and softmax is just the right activation method for that.

In [31]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 28, 128)           256000    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 28, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


Hereby I declare the train and test dataset.

In [32]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(7188, 28) (7188, 2)
(3541, 28) (3541, 2)


In [33]:
print(X_train[:3])

[[   0    0    0    0    0    0    0    0   72   11   80  312    7  980
   781  758  101   28   99  234  278   21  375  199  414    6    7  185]
 [   0    0    0    0    0    0    0    0    0    0    0    0   10   86
    15    7  345  104  182   29  359  125   63   15  354   13    2    3]
 [   0    0    0    0    0    0    0    0    0  422  553   91   19  414
    62   25  431 1036   83   40  673  508 1734  816  264  312   16  134]]


Here we train the Network. We should run much more than 7 epoch, but I would have to wait forever for kaggle, so it is 7 for now.

In [34]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 9, batch_size=batch_size, verbose = 2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/9
 - 20s - loss: 0.4405 - accuracy: 0.8144
Epoch 2/9
 - 20s - loss: 0.3251 - accuracy: 0.8639
Epoch 3/9
 - 20s - loss: 0.2833 - accuracy: 0.8830
Epoch 4/9
 - 20s - loss: 0.2516 - accuracy: 0.9001
Epoch 5/9
 - 20s - loss: 0.2272 - accuracy: 0.9065
Epoch 6/9
 - 20s - loss: 0.2074 - accuracy: 0.9174
Epoch 7/9
 - 20s - loss: 0.1855 - accuracy: 0.9260
Epoch 8/9
 - 20s - loss: 0.1692 - accuracy: 0.9329
Epoch 9/9
 - 20s - loss: 0.1622 - accuracy: 0.9359


Extracting a validation set, and measuring score and accuracy.

In [35]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.48
acc: 0.83


Finally measuring the number of correct guesses.  It is clear that finding negative tweets goes very well for the Network but deciding whether is positive is not really. My educated guess here is that the positive training set is dramatically smaller than the negative, hence the "bad" results for positive tweets.

In [36]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 58.89967637540453 %
neg_acc 90.76406381192275 %


As it was requested by the crowd, I extended the kernel with a prediction example, and also updated the API calls to Keras 2.0. Please note that the network performs poorly. Its because the training data is very unbalanced (pos: 4472, neg: 16986), you should get more data, use other dataset, use pre-trained model, or weight classes to achieve reliable predictions.

I have created this kernel when I knew much less about LSTM & ML. It is a really basic, beginner level kernel, yet it had a huge audience in the past year. I had a lot of private questions and requests regarding this notebook and I tried my best to help and answer them . In the future I am not planning to answer custom questions and support/enhance this kernel in any ways. Thank you my folks :)

In [41]:
twt = [' i am very sad ']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   9 158 100 725]]
negative
